# <center> Introdução ao Pandas - Parte 3 </center>
___

<p align="center">
  <img src="https://nsee.maua.br/wp-content/uploads/2023/05/logo_nsee_white.svg",width=200, height=120/>
</p>

___

## Conteúdo
1. [Pandas (Partes 2 e 3)](#pandas)
    1. [Carregando Variáveis](#carregando)
    2. [Juntando DataFrames](#merge)
    3. [Exercício 2.3](#ex2.3)
    4. [Alterando o dataFraame](#alterando)
    5. [Operações em grupo](#group)
    6. [Exercício 2.4](#ex2.4)
    7. [Aplicando funções no Pandas](#func)
    8. [Exercício 2.5](#ex2.5)

___

## <center> Objetivos de aprendizado </center>
- Familiarizar-se com as funcionalidades básicas do Pandas
- Ser capaz de carregar dados em um DataFrame
- Ser capaz de realizar manipulações básicas de dados
___


<a name="carregando"></a>

## Carregando Variáveis

Para conseguirmos seguir de onde paramos no notebook anterior, faremos a leitura de um arquivo **pickle** na célula a seguir. Não se preocupe em entender o significado da célula a seguir nesse momento. O arquivo ```datasets.pkl``` deve se encontrar junto com os arquivos de dados desse notebook. Basta colocá-lo na mesma pasta do seu notebook (caso esteja rodando pelo Jupyter) ou na raiz dos arquivos (caso esteja usando o Colab) e rodar a célula a seguir.

In [226]:
import pickle

with open("dados/datasets.pkl", "rb") as f:
    pkmn = pickle.load(f)
    fut_players = pickle.load(f)


<a name="merge"></a>
## 2.7. Juntando DataFrames

É muito comum ter a necessidade de juntar *DataFrames* diferentes. O Pandas tem a funcionalidade de juntar dataframes utilizando o método *.merge()*. Antes do exemplo, vamos aprender os tipos de *joins* mais comuns:<br>

![Joining Methods](https://i.imgur.com/HaSBT91.jpg) <br>


Agora, vamos carregar um DataFrame mais simples para testar os tipos de *merge*.

In [227]:
import pandas as pd

# Execute esta célula para carregar o dataframe metal_bands com dados de bandas de metal
metal_bands = pd.read_csv("https://raw.githubusercontent.com/pedrogjmesquita/python_notebooks/main/Pandas%20e%20Numpy/dados/metal_bands.csv", sep=",", encoding='Latin')
metal_bands.head(10)

,id,band_name,fans,formed,origin,split,style
0,0,Iron Maiden,4195,1975,United Kingdom,-,"New wave of british heavy,Heavy"
1,1,Opeth,4147,1990,Sweden,1990,"Extreme progressive,Progressive rock,Progressive"
2,2,Metallica,3712,1981,USA,-,"Heavy,Bay area thrash"
3,3,Megadeth,3105,1983,USA,1983,"Thrash,Heavy,Hard rock"
4,4,Amon Amarth,3054,1988,Sweden,-,Melodic death
5,5,Slayer,2955,1981,USA,1981,Thrash
6,6,Death,2690,1983,USA,2001,"Progressive death,Death,Progressive thrash"
7,7,Dream Theater,2329,1985,USA,1985,Progressive
8,8,Black Sabbath,2307,1968,United Kingdom,-,"Doom,Heavy,Hard rock"
9,9,Nightwish,2183,1996,Finland,1996,"Symphonic power,Gothic,Symphonic"


Podemos ver na célula acima, que podemos "importar" um dataframe diretamente de um URL, sem ter que baixar o arquivo. Isso é muito útil quando estamos trabalhando com arquivos muito grandes, que podem demorar para baixar.<br>

Assim como criamos os dataframes *offensive_stats*, *defensive_stats*, *fire_pkmn* e *water_pkmn*, vamos separar alguns dataframes a partir de *metal_bands* para testar os merges. Observe a célula abaixo.

In [228]:
bands_origin = metal_bands[['id','band_name','formed','origin']] # ano de formação e país das bandas
bands_style = metal_bands[['id','band_name','style']] # estilo das bandas

bands_split = metal_bands[metal_bands['split']!='-'][['id','band_name','split']] # bandas que se separaram
bands_4000_fans = metal_bands[metal_bands['fans']>4000][['id','band_name','fans']] # bandas com mais de 4000 fans
bands_USA = metal_bands[metal_bands['origin']=='USA'][['id','band_name','formed','origin']] # bandas formadas nos EUA
bands_Brazil = metal_bands[metal_bands['origin']=='Brazil'][['id','band_name','formed','origin']] # bandas formadas no Brasil

### 2.7.1. ``inner merge``

Vamos criar um DataFrame a partir de ```bands_origin``` e ```bands_split```, utilizando *merge*.

In [229]:
origin_split = pd.merge(
    bands_origin, # o DataFrame da esquerda
    bands_split, # o DataFrame da direita
    how='inner', # o tipo de join que queremos fazer
    on='id') # baseado em quais valores em comum (chave)
origin_split.info()
origin_split.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2785 entries, 0 to 2784
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           2785 non-null   int64 
 1   band_name_x  2785 non-null   object
 2   formed       2785 non-null   object
 3   origin       2782 non-null   object
 4   band_name_y  2785 non-null   object
 5   split        2785 non-null   object
dtypes: int64(1), object(5)
memory usage: 130.7+ KB


,id,band_name_x,formed,origin,band_name_y,split
0,1,Opeth,1990,Sweden,Opeth,1990
1,3,Megadeth,1983,USA,Megadeth,1983
2,5,Slayer,1981,USA,Slayer,1981
3,6,Death,1983,USA,Death,2001
4,7,Dream Theater,1985,USA,Dream Theater,1985


Ótimo! Conseguimos fazer o *merge* (termo mais utilizado no Pandas) de dois *DataFrames*. Observe que utilizamos o argumento ```how='inner'```. Lembre-se que *inner*, *left*, *right* e *outer* terão resultados diferentes, observe os merges abaixo e a explicação.

O *inner* mantém apenas os dados das bandas encontradas nos dois dataframes (onde há correspondência de *id*), dessa forma, a posição do dataframe não faz diferença. Pense no *inner* como uma interseção entre dois conjuntos, assim como visto em matemática, desta forma:

$$inner(A,B) = A \cap B$$

Portanto o DataFrame resultante ```origin_split``` é um dataframe que contem as informações de ```id```, ```nome```, ```ano de formação```, ```ano de separação```

### 2.7.2 ``left merge``

In [230]:
left_origin_split = pd.merge(bands_origin, bands_split, how='left', on='id')
left_origin_split.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           5000 non-null   int64 
 1   band_name_x  5000 non-null   object
 2   formed       5000 non-null   object
 3   origin       4992 non-null   object
 4   band_name_y  2785 non-null   object
 5   split        2785 non-null   object
dtypes: int64(1), object(5)
memory usage: 234.5+ KB


Como podemos ver com o resultado do método *.info()*, os resultados são de fato bem diferentes.

No *left*, como mostra no [diagrama](#merge) mantemos os dados do dataframe à esquerda, e trazemos os dados do dataframe à direita no qual encontrou-se a chave (neste exemplo, o *id* da banda). Quando fazemos o *left* damos uma *"importância"* maior aos dados do dataframe à esquerda. Assim:

$$left(A,B) = A \cup (A.key \cap B.key)$$

No nosso exemplo, quando fizemos o *left* criamos o ```left_origin_split```, isso nos dá um DataFrame com todas as informações de ```bands_origin``` e apenas as informações de ```bands_split``` cujas bandas estão presentes em ```bands_origin```. Neste exemplo, o datafram resultante é igual ao ```bands_origin```, porém com a coluna de *split*.

### 2.7.3 ``right merge``

In [231]:
right_origin_split = pd.merge(bands_origin, bands_split, how='right', on='id')
right_origin_split.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2785 entries, 0 to 2784
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           2785 non-null   int64 
 1   band_name_x  2785 non-null   object
 2   formed       2785 non-null   object
 3   origin       2782 non-null   object
 4   band_name_y  2785 non-null   object
 5   split        2785 non-null   object
dtypes: int64(1), object(5)
memory usage: 130.7+ KB



Por outro lado, no *right* ocorre o contrário, mantemos os dados do dataframe à direita e, quando há correspondência da chave, trazemos os dados do dataframe à esquerda. Assim:

$$right(A,B) = B \cup (B.key \cap A.key)$$

 Note que o número de entradas (*entries*) é diferente do caso com o *left*. Isso ocorre porque no *left* mantemos os dados de formação das bandas (ou seja, o dataframe contém todas as bandas do .csv), enquanto no *right*, mantemos apenas os dados de bandas que se separaram (e existem muitas bandas que ainda continuam juntas).


### 2.7.4 ``Outer Merge``

In [232]:
print('Numero de linhas do DataFrame bands_4000_fans:', bands_4000_fans.shape[0])
print('Numero de linhas do DataFrame bands_USA:', bands_USA.shape[0])
print('----------------------------------------------')
outer_origin_split = pd.merge(bands_4000_fans, bands_USA, how='outer', on='id')
outer_origin_split.info()

Numero de linhas do DataFrame bands_4000_fans: 4
Numero de linhas do DataFrame bands_USA: 1139
----------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1143 non-null   int64  
 1   band_name_x  4 non-null      object 
 2   fans         4 non-null      float64
 3   band_name_y  1139 non-null   object 
 4   formed       1139 non-null   object 
 5   origin       1139 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 53.7+ KB


Por fim, no *outer* utilizamos dois dataframes diferentes dos anteriores para facilitar o entendimento. Observe pelos prints que existem apenas 4 bandas com mais de 4000 fans e 1139 bandas formadas nos EUA. Quando fazemos o *merge* com *outer*, observe que o total de linhas passa a ser 1143. O que acontece é que esse tipo de join mantém os dados de ambos os dataframes, independente se houve correspondência de chave ou não. Dessa maneira:

$$outer(A,B) = A\cup B$$


### 2.7.5 Concatenação de DataFrames

Podemos também querer apenas concatenar dois *DataDrames*, isto é, juntá-los colocando um abaixo ou ao lado do outro, assim como fizemos com strings e listas nos primeiros notebooks. Para isso, utilizamos o método *.concat()*:

In [233]:
USA_Brazil = pd.concat([bands_USA, bands_Brazil], ignore_index=True) # concatenando bandas formadas nos EUA e bandas formadas no Brasil
USA_Brazil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1198 non-null   int64 
 1   band_name  1198 non-null   object
 2   formed     1198 non-null   object
 3   origin     1198 non-null   object
dtypes: int64(1), object(3)
memory usage: 37.6+ KB


Acima fizemos a concatenação vertical. Vamos fazer a horizontal abaixo:

In [234]:
bands_origin_style = pd.concat([bands_origin, bands_style], axis=1)
bands_origin_style.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         5000 non-null   int64 
 1   band_name  5000 non-null   object
 2   formed     5000 non-null   object
 3   origin     4992 non-null   object
 4   id         5000 non-null   int64 
 5   band_name  5000 non-null   object
 6   style      5000 non-null   object
dtypes: int64(2), object(5)
memory usage: 273.6+ KB


Você deve estar se perguntando: mas então qual a diferença entre utilizar o *merge* e o *concat* com axis=1 (concatenação horizontal)? Observe a imagem abaixo.

![concat](https://i.imgur.com/YlmiwsR.png) <br>

Note que o *concat* recebe os dataframes e apenas os empilha (verticalmente ou horizontalmente). Observe agora o funcionamento do *merge* na imagem abaixo.

![merge](https://i.imgur.com/yGum2id.png) <br>

Com o *merge*, podemos combinar os dataframes de acordo com os valores de suas colunas. Passamos a coluna a ser utilizada como chave, e os valores serão apenas combinados caso haja correspondência nos dois dataframes.

<a name="ex2.3"></a>
## Exercício 2.3
Mais uma vez, substitua os \____ de acordo com as instruções

In [235]:
# # the_best é um DataDrame dos melhores jogadores em drible (dribbling) e chute (shooting)
# the_best = fut_players[(fut_players.dribbling > 90) & (fut_players.shooting > 90)][['id', 'name', 'position', 'dribbling', 'shooting', 'overall']]

# # nacionalidades é um DataDrame da nacionalidade dos jogadores
# nacionalidades = fut_players[['id', 'name', 'nationality']]

# # faça um merge dos dois DataDrames, mantendo todos os jogadores de the_best e obtendo suas nacionalidades (dica: a chave é o id)
# the_best_nacionalidades = ____
# the_best_nacionalidades.head()

<a name="alterando"></a>
## 2.8. Alterando o dataframe

Até o momento apenas utilizamos os dados da forma que nos foram fornecidos, mas e se precisássemos criar alguma coluna que fosse a combinação das demais? Por exemplo, caso eu deseje criar uma coluna que corresponde à soma do ataque e velocidade dos Pokémons? Observe abaixo:

In [236]:
# Criando a coluna desejada
pkmn['AttackOverDefense'] = pkmn['Attack'] / pkmn['Defense']
pkmn.head()

,#,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Legendary,AttackOverDefense
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,1.000000
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,0.984127
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,0.987952
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,0.813008
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,1.209302


Observe como foi fácil! Apenas utilizamos o operador de soma com as duas colunas necessárias. Você pode fazer isso com outras operações também, basta utilizar ```+```, ```-``` ou ```*```. Além disso, você pode combinar quantas colunas quiser!

Mas e se precisarmos alterar apenas algumas linhas do nosso DataFrame?

Por exemplo, suponha que você percebeu que seus dados estão errados, e todos os Pokémons com velocidade acima de 100 deveriam estar marcados como Type_1 = 'Fire', podemos seguir o procedimento abaixo:

In [237]:
# Observe os valores unicos da coluna Type_1 para os Pokémons com mais de 100 de velocidade
pkmn.loc[pkmn['Speed']>100, 'Type_1'].unique()

array(['Bug', 'Normal', 'Electric', 'Ground', 'Psychic', 'Fire', 'Ghost',
       'Water', 'Rock', 'Poison', 'Grass', 'Dark', 'Dragon', 'Steel',
       'Fighting', 'Ice', 'Flying'], dtype=object)

In [238]:
# Vamos alterar tudo para Fire
pkmn.loc[pkmn['Speed']>100, 'Type_1'] = 'Fire'

In [239]:
# Observe como os valores mudaram
pkmn.loc[pkmn['Speed']>100, 'Type_1'].unique()

array(['Fire'], dtype=object)

Antes de continuar, vamos ler novamente os dados de Pokémon, sem essa última alteração. Execute a célula abaixo.

In [240]:
pkmn = pd.read_csv(
    'dados/pokemon_data.csv', # o caminho para o arquivo que se quer ler
    sep=',') # o caracter utilizado para separar os valores

pkmn.rename(
    columns={'Type 1':'Type_1', 'Type 2':'Type_2', 'Sp. Atk':'Sp_Atk','Sp. Def':'Sp_Def'}, # passando o nome antigo e novo como um dicionário
    inplace = True # algumas operações com Pandas criam uma cópia do DataFrame e não alteram o objeto em si, alteramos isso mudando o parâmetro inplace para verdadeiro
)

<a name="group"></a>
## 2.9. Operações em grupo

Com Pandas nós podemos aplicar operações em grupos usando o método *.groupby()*. Ele é muito útil por ser uma forma bem simples de extrair informação de dados agregados. Para utilizá-lo, passamos as colunas nas quais queremos agrupar os dados e a operação que queremos fazer. Para exemplificar, vamos ver quantos Pokémons lendários cada geração tem:

In [241]:
pkmn.groupby('Generation').Legendary.sum() # fazendo uma soma pois a coluna Legendary é boolean

Generation
1     6
2     5
3    18
4    13
5    15
6     8
Name: Legendary, dtype: int64

Podemos obter um relatório da média de diversas colunas para cada tipo de Pokémon:

In [242]:
pkmn.groupby('Type_1')[['HP','Attack','Defense','Sp_Atk','Sp_Def','Speed']].mean()

,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed
Type_1,,,,,,
Bug,56.884058,70.971014,70.724638,53.869565,64.797101,61.681159
Dark,66.806452,88.387097,70.225806,74.645161,69.516129,76.161290
Dragon,83.312500,112.125000,86.375000,96.843750,88.843750,83.031250
Electric,59.795455,69.090909,66.295455,90.022727,73.704545,84.500000
Fairy,74.117647,61.529412,65.705882,78.529412,84.705882,48.588235
Fighting,69.851852,96.777778,65.925926,53.111111,64.703704,66.074074
Fire,69.903846,84.769231,67.769231,88.980769,72.211538,74.442308
Flying,70.750000,78.750000,66.250000,94.250000,72.500000,102.500000
Ghost,64.437500,73.781250,81.187500,79.343750,76.468750,64.343750


Isso é realmente muito importante e extremamente utilizado com pandas pois conseguimos fazer análises dos grupos com apenas uma linha de código. Podemos perceber, por exemplo, que Pokémons do tipo *Flying* são especialistas em velocidade enquanto *Dragon* e *Fighting* são especialistas em ataque.

<a name="ex2.4"></a>
## Exercício 2.4
Use o método *.groupby()* para descobrir qual país tem o melhor *overall* médio.

In [243]:
# # crie o DataFrame country_avg_overall, que tem o overall médio de cada país (nationality), usando groupby
# country_avg_overall = ____

# # usamos o método idxmax() para encontrar o maior overall médio
# printf("Melhor overall médio: {country_avg_overall.loc[country_avg_overall.idxmax()]}")
# printf("Overall médio do Brasil: {country_avg_overall.loc['Brazil']}")

<a name="func"></a>
### 2.10. Aplicando funções no Pandas

Com Pandas, nós temos um grande nível de controle de nossos dados, e somos capazes de transformá-los conforme precisarmos. Nós podemos, até mesmo, executar funções em DataFrames e manipulá-los como quisermos. Vamos revisitar o método head():

In [244]:
pkmn.head()

,#,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


Existem algumas mega evoluções misturadas no dataset (apenas alguns Pokémons são capazes de evoluir temporariamente para sua forma Mega, uma forma mais poderosa). Não seria legal se nós tivéssemos alguma flag que nos diria se um pokémon é mega ou não? E, por um acaso, será que os pokémons mega são mais poderosos?

Você deve ter percebido que evoluções mega têm um padrão em nosso DataFrame, algo como 'PokemonMega Pokemon'. Se nós tivermos esse padrão, podemos construir uma função que retorna True se este padrão for detectado:

In [245]:
def ehMega(pokemon_nome):
    """
    Recebe um nome de pokemon e diz se é uma mega evolução ou não
    I: string pokemon_nome
    O: boolean para Mega evos
    """
    if 'Mega ' in pokemon_nome: # é importante usar 'Mega ' e não 'mega', pois há um pokemon chamado Yanmega e outro chamado Meganium que não são uma mega evolução
        return True
    else:
        return False

Vamos ver se funciona:

In [246]:
ehMega('VenusaurMega Venusaur')

True

In [247]:
ehMega('Squirtle')

False

Excelente! Seria ótimo se conseguíssemos aplicar essa função em todo nosso DataFrame. Para fazer isso, usaremos o método .apply(). Também criaremos uma coluna que é uma flag se o pokémon é mega:

In [248]:
pkmn['Mega'] = pkmn.apply(
    lambda linha: ehMega(linha['Name']), # chamando uma função lambda que acabamos de construir
    axis=1 # qual direção queremos executar a função. 0 para horizontal, 1 para vertical
)

Funções ```lambda``` é uma funcionalidade do Python que nos permite criar funções em uma linha de forma mais fácil. Não falaremos muito sobre essas funções, mas ela é muito boa para se ter no seu arsenal de Cientista de Dados. Se você quiser saber mais, [clique aqui](https://www.w3schools.com/python/python_lambda.asp).

In [249]:
pkmn.head()

,#,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Legendary,Mega
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,True
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,False


Observe como funcionou o apply com lambda. Utilizando o axis=1, a função ```ehMega``` é aplicada para cada linha do nosso DataFrame, recebendo como entrada o *Name* do pokémon daquela linha e retornando a flag de True/False na coluna *Mega*.

Agora, vamos verificar quão poderosos são os pokémons mega:

In [250]:
pkmn.groupby('Mega').Total.mean()

Mega
False    423.457447
True     617.541667
Name: Total, dtype: float64

Eles têm quase 200 stat points a mais que pokémons normais! Evoluções mega são, sim, muito poderosos! 

Uma boa prática é sempre tentar manter nosso DataFrame organizado. A forma como os pokémons mega estão nomeados não é muito ótima, e nós já temos uma coluna com a flag para pokémons Mega, então, vamos atacar isso! A estrutura do nome de um pokémon mega é da seguinte forma: 'NomeMega Nome'. Portanto, se nós pegarmos o que vem após o caractere ' ', teremos o nome original do pokémon!

In [251]:
pkmn.Name.nunique() # conta elementos únicos de uma determinada coluna

800

In [252]:
def get_nome_original(nome):
    """
    Recebe um nome de pokemon e retorna seu nome original
    I: nome string
    O: string
    """
    return nome.split(' ')[0]


Vamos agora usar a função que acabamos de criar para sobreescrever a coluna ``Name`` do nosso DataFrame

In [253]:

pkmn['Name'] = pkmn.Name.apply(lambda s: get_nome_original(s)) # sobreescrevendo a coluna Name
pkmn.Name.nunique()

797

In [254]:
pkmn.head()

,#,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Legendary,Mega
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,False
3,3,VenusaurMega,Grass,Poison,625,80,100,123,122,120,80,1,False,True
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,False


Agora nós já cobrimos toda a parte básica de Pandas! Vamos praticar essa última parte!

<a name="ex2.5"></a>
### Exercício 2.5
Crie uma função que retorna a classificação para o jogador de acordo com as instruções abaixo, então aplique isso para o dataframe fut_players.

*Observação:* considere os limites dentro do intervalo de classificação.
exemplo

-50 contém todos os valores menores que 50 e o valor 50 incluso;


51-60 contém todos os valores entre 51 e 60 com os limites [51,60] inclusos no grupo;


e assim por diante ...

In [255]:
def get_classificacao(overall):
    """
    Recebe um overall de algum jogador e retorna a classificação conforme a seguir:
    Overall -> classificação
    -50     -> "Amador"
    51-60   -> "Ruim"
    61-70   -> "Ok"
    71-80   -> "Bom"
    81-90   -> "Ótimo"
    91+     -> "Lenda"
    
    I: int overall
    O: string
    """
    ____
    
fut_players["classification"] = ____
fut_players.groupby('classification')[['id']].count()

NameError: name '____' is not defined

# Fim da aula!

Obrigado por participar do curso, você acaba de finalizar a aula de Python e Pandas. Neste momento você já deve ser capaz de manipular seus dados no Python, utilizando as bibliotecas que acabamos de aprender! 

Lembre-se que sempre que surgir alguma dúvida, você pode olhar a documentação do [NumPy](https://numpy.org/doc/) e do [Pandas](https://pandas.pydata.org/docs/).